## Imports and Initialization of script

In [1]:
!pip install pyspark

import sys
import os

os.environ['HADOOP_HOME'] = "C:\Hadoop"
sys.path.append("C:\Hadoop\bin")

os.environ['JAVA_HOME'] = "C:\Java"
sys.path.append("C:\Java\bin")

In [2]:
from pyspark.sql.functions import sum,avg,max,count, round, col, asc, DataFrame, expr, split, lit
from pyspark.sql import Row

from functools import reduce 

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.\
builder.\
appName("pyspark-notebook2").\
master("local").\
config("spark.executor.memory", "1g").\
config("spark.mongodb.input.uri","mongodb://127.0.0.1/Project6").\
config("spark.mongodb.output.uri","mongodb://127.0.0.1/Project6").\
config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
config("spark.jars", "\Spark\spark-3.3.1-bin-hadoop3\jars\mysql-connector-j-8.0.32.jar").\
getOrCreate()

## Extract

In [4]:
collection_list = ['Research_2007_2013', 'Research_2007_2013_org', 'Research_2007_2013_themes',\
                  'Research_2014_2020', 'Research_2014_2020_org', 'Research_2014_2020_themes',\
                  'Research_2021_2027', 'Research_2021_2027_org', 'Research_2021_2027_themes',\
                  'OCDE_ind']
    
def extract_collection(collection):
    spark.conf.set('spark.sql.caseSensitive', True)
    
    df_collection = spark.read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://localhost:27017/Project6") \
    .option("database", "Project6") \
    .option("collection", collection) \
    .load()
    return df_collection

result = list(map(extract_collection, collection_list))

Py4JJavaError: An error occurred while calling o43.load.
: com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=localhost:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.ConnectException: Connection refused: connect}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:177)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:41)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:147)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:98)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.getClientSession(MongoClientDelegate.java:278)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperat

In [ ]:
#check the size of the df's
print(collection_list[0], 'has ',result[0].count(), 'rows and ', len(result[0].columns), 'columns')
print(collection_list[1], 'has ',result[1].count(), 'rows and ', len(result[1].columns), 'columns')
print(collection_list[2], 'has ',result[2].count(), 'rows and ', len(result[2].columns), 'columns')
print(collection_list[3], 'has ',result[3].count(), 'rows and ', len(result[3].columns), 'columns')
print(collection_list[4], 'has ',result[4].count(), 'rows and ', len(result[4].columns), 'columns')
print(collection_list[5], 'has ',result[5].count(), 'rows and ', len(result[5].columns), 'columns')
print(collection_list[6], 'has ',result[6].count(), 'rows and ', len(result[6].columns), 'columns')
print(collection_list[7], 'has ',result[7].count(), 'rows and ', len(result[7].columns), 'columns')
print(collection_list[8], 'has ',result[8].count(), 'rows and ', len(result[8].columns), 'columns')

In [ ]:
# list_x = [0, 1, 2]

# def printing(x):
#     print (collection_list[x], 'has ',result[x].count(), 'rows and ', len(result[x].columns), 'columns')
    
# list(map(printing(list_x)))

## Transform

In [ ]:
#drop columns

In [ ]:
#drop columns on MAIN research dataframes
columns_main_07_13 = ('acronym', 'status', 'legalBasis', 'topics', 'subCall', 'fundingScheme', 'contentUpdateDate', 'nature')
columns_main_other = ('acronym', 'status', 'legalBasis', 'topics', 'subCall', 'fundingScheme', 'contentUpdateDate', 'ecSignatureDate', 'grantDoi', 'masterCall', 'nature')

research_2007_2013 = result[0].drop(*columns_main_07_13)
research_2014_2020 = result[3].drop(*columns_main_other)
research_2021_2027 = result[6].drop(*columns_main_other)

#check number of columns
print('The number of columns before drop were ', len(result[0].columns), 'and now are ', len(research_2007_2013.columns))
print('The number of columns before drop were ', len(result[3].columns), 'and now are ', len(research_2014_2020.columns))
print('The number of columns before drop were ', len(result[6].columns), 'and now are ', len(research_2014_2020.columns))

In [ ]:
#drop columns on ORG research dataframes
columns_org_07_13 = ('projectAcronym', 'vatNumber', 'street', 'postCode', 'organizationURL', 'contactForm', 'contentUpdateDate', 'endOfParticipation')
columns_org_other = ('projectAcronym', 'vatNumber', 'street', 'postCode', 'organizationURL', 'contactForm', 'contentUpdateDate', 'endOfParticipation', 'SME', 'active', 'totalCost', 'netEcContribution')

research_2007_2013_org = result[1].drop(*columns_org_07_13)
research_2014_2020_org = result[4].drop(*columns_org_other)
research_2021_2027_org = result[7].drop(*columns_org_other)

#check number of columns
print('The number of columns before drop were ', len(result[1].columns), 'and now are ', len(research_2007_2013_org.columns))
print('The number of columns before drop were ', len(result[4].columns), 'and now are ', len(research_2014_2020_org.columns))
print('The number of columns before drop were ', len(result[7].columns), 'and now are ', len(research_2014_2020_org.columns))

In [ ]:
#merge all main research df's

df_merge = research_2007_2013.unionAll(research_2014_2020)
df_research_main = df_merge.unionAll(research_2021_2027)

In [ ]:
#create new column ecContribution% on research main

df_research_main.withColumn("% EcContibution", df_research_main.ecMaxContribution*100/df_research_main.totalCost)

In [ ]:
#create new column start year

df_research_main = df_research_main.withColumn('start_year', split(df_research_main['startDate'], '-').getItem(0)) \
       .withColumn('end_year', split(df_research_main['endDate'], '-').getItem(1))

In [ ]:
#merge all org research df's

df_merge1 = research_2007_2013_org.unionAll(research_2014_2020_org)
df_research_org = df_merge1.unionAll(research_2021_2027_org)

In [ ]:
#create new column activityType_name on research org

df_research_org = df_research_org.withColumn("activityType_name", expr("CASE WHEN activityType = 'HES' THEN 'Education Establishments' " + 
               "WHEN activityType = 'REC' THEN 'Research Organisations' WHEN activityType = 'PRC' THEN 'Private sector' \
               WHEN activityType = 'PRC' THEN 'Private sector' \
               WHEN activityType = 'PUB' THEN 'Public bodies' \
               WHEN activityType = 'OTH' THEN 'Other' \
               WHEN activityType IS NULL THEN 'Not_considered'" +\
               "ELSE activityType END"))

In [ ]:
#Split column geolocation on research org

df_research_org = df_research_org.withColumn('latitude', split(df_research_org['geolocation'], ',').getItem(0)) \
       .withColumn('longitude', split(df_research_org['geolocation'], ',').getItem(1))

In [ ]:
#merge all themes research df's

#check column names
print(result[2].schema.names)
print(result[5].schema.names)
print(result[8].schema.names)
#ok to merge

In [ ]:
df_merge2 = result[2].unionAll(result[5])
df_research_themes = df_merge2.unionAll(result[8])

In [ ]:
#Split column path in THEMES research dataframes
cols_to_drop = ('to_drop', 'sub_sub_path')


df_research_themes = df_research_themes.withColumn('to_drop', split(df_research_themes['euroSciVocPath'], '/').getItem(0)) \
        .withColumn('main_path', split(df_research_themes['euroSciVocPath'], '/').getItem(1)) \
        .withColumn('sub_path', split(df_research_themes['euroSciVocPath'], '/').getItem(2)) \
        .withColumn('sub_sub_path', split(df_research_themes['euroSciVocPath'], '/').getItem(3)) \
        .drop(*cols_to_drop)

In [ ]:
#merge all dfs
df_research_main_org = df_research_org.join(df_research_main,df_research_org["projectID"] == df_research_main["id"])

In [ ]:
df_research_all = df_research_main_org.join(df_research_themes,df_research_main_org["projectID"] == df_research_themes["projectID"]).drop('_id', 'projectID', 'rcn')

In [ ]:
# df_time_sum = (df_OCDE_ind.groupby('TIME')\
# .agg(sum('Value')\
# .alias('sum'))\
# .sort(col('TIME')\
# .desc()))

# df_time_sum.show()

In [ ]:
df_research_all.schema.names

In [ ]:
#drop rows where nans

print('Before drop start_year: ', df_research_all.count())
df_research_all = df_research_all.na.drop(subset=['start_year'])
print('After drop start_year: ', df_research_all.count())

print('Before drop country: ', df_research_all.count())
df_research_all = df_research_all.na.drop(subset=['country'])
print('After drop country: ', df_research_all.count())


In [ ]:
#verify country codes
(df_research_all.groupby('country')
.agg(count('id')
.alias('count'))
.sort(col('count')
.desc())
.show(50))


In [ ]:
# #rename country codes

# df_research_all = df_research_all.withColumn("country_name", expr("CASE WHEN country = 'DE' THEN 'Germany' " + 
#                "WHEN country = 'UK' THEN 'United Kingdom' \
#                WHEN country = 'ES' THEN 'Spain' \
#                WHEN country = 'FR' THEN 'France' \
#                WHEN country = 'IT' THEN 'Italy' \
#                WHEN country = 'NL' THEN 'Netherlands' \
#                WHEN country = 'BE' THEN 'Belgium' \
#                WHEN country = 'CH' THEN 'Switzerland' \
#                WHEN country = 'EL' THEN 'Greece' \
#                WHEN country = 'SE' THEN 'Sweden' \
#                WHEN country = 'AT' THEN 'Austria' \
#                WHEN country = 'DK' THEN 'Denmark' \
#                WHEN country = 'PT' THEN 'Portugal' \
#                WHEN country = 'FI' THEN 'Finland' \
#                WHEN country = 'NO' THEN 'Norway' \
#                WHEN country = 'IE' THEN 'Ireland' \
#                WHEN country = 'PL' THEN 'Poland' \
#                WHEN country = 'IL' THEN 'Israel' \
#                WHEN country = 'CZ' THEN 'Czech Republic' \
#                WHEN country = 'US' THEN 'United States' \
#                WHEN country IS NULL THEN 'Not_considered'" +\
#                "ELSE activityType END"))

## Load

In [ ]:
# df_research_all.write \
#   .format("jdbc") \
#   .option("driver","com.mysql.cj.jdbc.Driver") \
#   .option("url", "jdbc:mysql://localhost:3306/Project6") \
#   .option("dbtable", "df_research_all") \
#   .option("user", "root") \
#   .option("password", "M1a2r3t4a5!") \
#   .mode('overwrite') \
#   .save()

In [ ]:
# df_research_all.write \
#   .format("jdbc") \
#   .option("driver","com.mysql.cj.jdbc.Driver") \
#   .option("url", "jdbc:mysql://localhost:3306/Project6") \
#   .option("dbtable", "df_research_all_new_new") \
#   .option("user", "root") \
#   .option("password", "M1a2r3t4a5!") \
#   .save()